## Name: Shivkumar G. Chauhan
### Roll No.: CS8A53
### Batch A-3
---
# Experiment - 3 : Data Preprocessing


## imports

In [1]:
import os
import pandas
import numpy
import matplotlib.pyplot as pyplot
import seaborn
import nltk
from nltk.corpus import stopwords,wordnet
from nltk.stem import WordNetLemmatizer
from nltk.probability import FreqDist

import re


import mysql.connector
import pymongo

## Configurations

In [2]:
# %matplotlib qt;
%matplotlib inline
pyplot.rcParams["figure.figsize"] = (15, 12);

In [3]:
Regular_expression_definition_for_html_tags=re.compile('<.*?>')
Regular_expression_definition_for_digits=re.compile('\d+\s|\s\d+|\s\d+\s')
Regular_expression_definition_for_links=re.compile('http://\S+|https://\S+')
word_lemitizer=WordNetLemmatizer()
english_stop_words=stopwords.words('english')

In [4]:
def getWordList(textList:list)->list:
    Words_list=[text for text in textList]
    Words_list=" ".join(Words_list)
    Words_list=Words_list.split()
    return Words_list

In [5]:
def cleaning_of_sentence(text):
    text = re.sub(r"http\S+|www\S+|https\S+", '', text, flags=re.MULTILINE)
    text=Regular_expression_definition_for_html_tags.sub(r" ",text)
    text=Regular_expression_definition_for_digits.sub(r" ",text)
    text=Regular_expression_definition_for_links.sub(r" ",text)
    punctuations = [".","=","_","<",">",",","!","?","'",'"',"’",":",";","*","-","/","+","%","&","$","#","@","(",")","[","]","{","}",'\n']
    for i in punctuations:
        text = text.replace(i," ")
    text=text.lower().split()
    text=[word_lemitizer.lemmatize(word) for word in text]
    text=[word for word in text if len(word)>1]
    return text

In [6]:
def getLikePerViews(list_of_likes,list_of_views):
    ratio=[]
    for i in range(len(list_of_likes)):
        if list_of_views[i]>=1:
            ratio.append(list_of_likes[i]/list_of_views[i])
        else:
            ratio.append(0)
    return ratio

### Data Loading

In [7]:
Himalaya_Tweets_Data=pandas.read_csv("../Data/Raw_Data/Twitter_Data/Himalaya_Tweets_Data.csv")

In [8]:
Tweets_Data_from_Search=pandas.read_csv("../Data/Raw_Data/Twitter_Data/Tweets_Data_from_Search.csv")

In [9]:
Himalaya_Wellness_Site_Products_Data=pandas.read_csv("../Data/Raw_Data/Himalaya_Wellness_Site_Data/Products_Data.csv")

In [10]:
Himalaya_Wellness_Site_Blog_Data=pandas.read_csv("../Data/Raw_Data/Himalaya_Wellness_Site_Data/Blogs_Data.csv")

In [11]:
Facebook_Post_Data=pandas.read_csv("../Data/Raw_Data/Facebook_Data/Facebook_Post_Data.csv")

In [12]:
Himalaya_Youtube_Video_Data=pandas.read_csv("../Data/Raw_Data/Youtube_Data/Himalaya_Youtube_Video_Data.csv")

In [13]:
Himalaya_Youtube_Video_Search_Data=pandas.read_csv("../Data/Raw_Data/Youtube_Data/Himalaya_Youtube_Video_Search_Data.csv")

In [14]:
Youtube_Comments_Data=pandas.read_csv("../Data/Raw_Data/Youtube_Data/Youtube_Comments_Data.csv")

### Credentials Loading

In [15]:
with open('./MySQL.env') as file:
    credentials=file.read()
credentials=credentials.split(" ")

In [16]:
connection_url=""
with open("./MongoDB.env") as file:
    connection_url=file.read()

### MySQL Connection

In [18]:
mysql_connection=mysql.connector.connect(username=credentials[0],password=credentials[1],host=credentials[2],port=int(credentials[3]))
queryExecuter=mysql_connection.cursor()

In [19]:
mysql_connection

In [ ]:
mysql_connection.close()

### MongoDB Connection

In [20]:
mongodb_connection=pymongo.MongoClient(connection_url)

In [21]:
mongodb_connection

MongoClient(host=['ac-xe1d4nk-shard-00-02.bew2itj.mongodb.net:27017', 'ac-xe1d4nk-shard-00-00.bew2itj.mongodb.net:27017', 'ac-xe1d4nk-shard-00-01.bew2itj.mongodb.net:27017'], document_class=dict, tz_aware=False, connect=True, authsource='admin', replicaset='atlas-fi3jx5-shard-0', tls=True)

## Data Exploration

### Initial Lookup

In [11]:
Himalaya_Tweets_Data.head()

,date,content,username,likes,views,hashtags,url
0,2023-01-25,The #HimalayaWellnessCompany commemorates the ...,HimalayaIndia,13,1052.0,"['HimalayaWellnessCompany', 'RepublicDay', 'Ha...",https://twitter.com/HimalayaIndia/status/16181...
1,2023-01-23,Himalaya’s new manufacturing facility in Tumak...,HimalayaIndia,20,1284.0,"['ISO', 'safety', 'HimalayaWellness']",https://twitter.com/HimalayaIndia/status/16173...
2,2023-01-17,"@MKC1_Reddy Hi Reddy, Could you please DM us y...",HimalayaIndia,0,29.0,NaN,https://twitter.com/HimalayaIndia/status/16152...
3,2023-01-13,Let's celebrate the onset of warmer days and p...,HimalayaIndia,3,473.0,"['HimalayaWellnessCompany', 'Lohri', 'Pongal',...",https://twitter.com/HimalayaIndia/status/16138...
4,2023-01-13,#Himalaya drives initiatives to enrich lives. ...,HimalayaIndia,3,679.0,"['Himalaya', 'Government', 'School']",https://twitter.com/HimalayaIndia/status/16137...


In [12]:
Tweets_Data_from_Search.head()

,date,content,username,likes,views,hashtags,url
0,2023-02-12,@Caudalie\n@HimalayaIndia \n@sleepy\n@babyturc...,rabiaerTURKc1,0,61,NaN,https://twitter.com/rabiaerTURKc1/status/16248...
1,2023-02-12,@PTI_News @PypAyurved @HimalayaIndia @DaburInd...,Green_Laalten,0,183,NaN,https://twitter.com/Green_Laalten/status/16245...
2,2023-02-10,Please declare these products @BhagwantMann g...,azeemali3119,2,115,"['Punjab', 'Ayurveda', 'unani', 'patanjalifood...",https://twitter.com/azeemali3119/status/162389...
3,2023-02-08,The prices of #soap are rising like anything i...,AmericanLove4u,0,58,"['soap', 'India', 'AUSvsIND', 'Cricket', 'INDv...",https://twitter.com/AmericanLove4u/status/1623...
4,2023-02-08,Shop @HimalayaIndia 🍃 Herbal products from @iH...,ayurvedanelly,0,49,"['India', 'himalaya', 'HimalayaIndia', 'Himala...",https://twitter.com/ayurvedanelly/status/16232...


In [13]:
Himalaya_Wellness_Site_Products_Data.head()

,title,price,description,url
0,Ashvagandha,₹ 200.00,Helps Boost Immunity and Overall Endurance \nF...,https://himalayawellness.in/collections/all/pr...
1,Gokshura,₹ 220.00,Improves vigour,https://himalayawellness.in/collections/all/pr...
2,Tentex Royal,₹ 225.00,Relieves stress & improves performance.\nHimal...,https://himalayawellness.in/collections/all/pr...
3,Tentex Forte,₹ 90.00,Rejuvenate & elevate your performance\nHimalay...,https://himalayawellness.in/collections/all/pr...
4,Complete Care Toothpaste,₹ 55.00,Healthy gums . Strong teeth . Fresh breath\nHi...,https://himalayawellness.in/collections/all/pr...


In [16]:
Himalaya_Wellness_Site_Blog_Data.head()

,name,posting_timestamp,content,url
0,Face gel over Face cream - Top 4 benefits to m...,08/10/2022 8:33am,Face gel is becoming increasingly popular in s...,https://himalayawellness.in/a/blog/benefits-fa...
1,Himalaya Immunity Boosters (Tablets and Syrups),07/11/2022 8:28am,"Adopting a healthy and balanced diet, exercisi...",https://himalayawellness.in/a/blog/himalaya-im...
2,8 Simple Steps for a Perfect Baby Massage,01/02/2023 10:40am,Are you wondering how to give your baby a soot...,https://himalayawellness.in/a/blog/steps-perfe...
3,10 Benefits of Guduchi (Giloy) Herb,01/02/2023 8:53am,Guduchi (Giloy) is an herb that has several th...,https://himalayawellness.in/a/blog/benefits-of...
4,How to Boost Bone Health with Nutrition,10/18/2022 8:37am,Osteoporosis (OP) is one of the most common me...,https://himalayawellness.in/a/blog/boost-bone-...


In [17]:
Facebook_Post_Data.head()

,posted_by,content,likes
0,MGB Foundation,Elevate your self-care routine with HimalayaGi...,3.0
1,Himalaya Face Care Nepal,Himalaya Purifying Neem Face Wash enriched wit...,201.0
2,Bestowic,If you're looking for a natural and effective ...,2.0
3,Plant Care Today,Pink Banana: How To Grow and Care For Musa Vel...,8.0
4,Shikhar Travels,Looking for an unforgettable adventure? Join u...,1.0


In [18]:
Himalaya_Youtube_Video_Data.head()

,link,title,duration,views,description,upload_date,category
0,https://www.youtube.com/watch?v=DAIFzQy3DMs,Himalaya Youth Eternity Face Sheet Mask | Heal...,20,129,Himalaya Youth Eternity Face Sheet Mask are in...,2023-02-17,Howto & Style
1,https://www.youtube.com/watch?v=vvXGQWoLA7Q,Himalaya Youth Eternity Face Sheet Mask | Heal...,50,191,Himalaya Youth Eternity Face Sheet Mask are in...,2023-02-17,Howto & Style
2,https://www.youtube.com/watch?v=Et8IRYf_iS0,Complete gum care protection,11,47165,Give your gums a complete natural care with he...,2023-01-23,Howto & Style
3,https://www.youtube.com/watch?v=VogJG0BsbQw,Himalaya Ayurveda Gum Care,10,15210,Wipe out gum problems with goodness of nature!...,2023-01-23,Howto & Style
4,https://www.youtube.com/watch?v=RFpxEatO5r0,Cure bleeding gums with Himalaya Ayurveda Gum ...,10,67714,Cure bleeding from the roots!\nHimalaya ayurve...,2023-01-23,Howto & Style


In [19]:
Himalaya_Youtube_Video_Search_Data.head()

,link,title,duration,views,description,upload_date,category
0,https://www.youtube.com/watch?v=0kZ-YTBxgFY,Himalaya face wash I Benefits,136,238,Himalaya face wash I Benefits\n\nHimalaya face...,2023-02-13,Entertainment
1,https://www.youtube.com/watch?v=Wvrl23CRhw0,Himalaya Moisturizing ALEO VERA gel | Product ...,236,60,Himalaya Moisturizing ALEO VERA gel\nFeatures ...,2023-02-11,People & Blogs
2,https://www.youtube.com/watch?v=whlQ2oI3Q9M,Best Baby 👣👶 care products Review for all moth...,351,187,Baby care Products @FashionandfoodHub \njohnso...,2023-01-30,People & Blogs
3,https://www.youtube.com/watch?v=K2IpqAOPblI,Top 12 best Himalaya skincare and haircare pro...,1474,117,#shri. #himalaya #skincare #haircare ...,2023-01-23,People & Blogs
4,https://www.youtube.com/watch?v=emqJpf29V1A,बिना पैसे खर्च किए निखार लाय/ Best winter Care...,177,63,बिना पैसे खर्च किए निखार लाय/ Face Care Produc...,2022-12-09,People & Blogs


In [20]:
Youtube_Comments_Data.head()

,video_link,author,content,likes
0,https://www.youtube.com/watch?v=0kZ-YTBxgFY,jibin c j,super,NaN
1,https://www.youtube.com/watch?v=whlQ2oI3Q9M,@rukhsarhabib1387,Thank you for sharing such precious piece of i...,1.0
2,https://www.youtube.com/watch?v=whlQ2oI3Q9M,@habibacheema2922,Amazing and great efforts,NaN
3,https://www.youtube.com/watch?v=whlQ2oI3Q9M,@Roshni7292,Ma Sha allha dear zabrdast information thanks,1.0
4,https://www.youtube.com/watch?v=whlQ2oI3Q9M,@saminarazalifestyle,Mashallah Zabardast Work Nice Vlog,NaN


In [21]:
Himalaya_Tweets_Data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   date      100 non-null    object 
 1   content   100 non-null    object 
 2   username  100 non-null    object 
 3   likes     100 non-null    int64  
 4   views     10 non-null     float64
 5   hashtags  75 non-null     object 
 6   url       100 non-null    object 
dtypes: float64(1), int64(1), object(5)
memory usage: 5.6+ KB


In [22]:
Tweets_Data_from_Search.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 127 entries, 0 to 126
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   date      127 non-null    object
 1   content   127 non-null    object
 2   username  127 non-null    object
 3   likes     127 non-null    int64 
 4   views     127 non-null    int64 
 5   hashtags  22 non-null     object
 6   url       127 non-null    object
dtypes: int64(2), object(5)
memory usage: 7.1+ KB


In [23]:
Himalaya_Wellness_Site_Products_Data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 885 entries, 0 to 884
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   title        885 non-null    object
 1   price        882 non-null    object
 2   description  860 non-null    object
 3   url          885 non-null    object
dtypes: object(4)
memory usage: 27.8+ KB


In [24]:
Himalaya_Wellness_Site_Blog_Data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20 entries, 0 to 19
Data columns (total 4 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   name               20 non-null     object
 1   posting_timestamp  20 non-null     object
 2   content            20 non-null     object
 3   url                20 non-null     object
dtypes: object(4)
memory usage: 768.0+ bytes


In [25]:
Facebook_Post_Data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 66 entries, 0 to 65
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   posted_by  65 non-null     object 
 1   content    63 non-null     object 
 2   likes      63 non-null     float64
dtypes: float64(1), object(2)
memory usage: 1.7+ KB


In [26]:
Himalaya_Youtube_Video_Data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   link         100 non-null    object
 1   title        100 non-null    object
 2   duration     100 non-null    int64 
 3   views        100 non-null    int64 
 4   description  100 non-null    object
 5   upload_date  100 non-null    object
 6   category     100 non-null    object
dtypes: int64(2), object(5)
memory usage: 5.6+ KB


In [27]:
Himalaya_Youtube_Video_Search_Data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 175 entries, 0 to 174
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   link         175 non-null    object
 1   title        175 non-null    object
 2   duration     175 non-null    int64 
 3   views        175 non-null    int64 
 4   description  168 non-null    object
 5   upload_date  175 non-null    object
 6   category     175 non-null    object
dtypes: int64(2), object(5)
memory usage: 9.7+ KB


In [28]:
Youtube_Comments_Data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150 entries, 0 to 149
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   video_link  150 non-null    object 
 1   author      150 non-null    object 
 2   content     150 non-null    object 
 3   likes       86 non-null     float64
dtypes: float64(1), object(3)
memory usage: 4.8+ KB


# Data Preprocessing

## 1. Data Cleaning

### Handling Missing Values

#### Searching for Missing Value

In [22]:
Himalaya_Tweets_Data.isnull().sum()

date         0
content      0
username     0
likes        0
views       90
hashtags    25
url          0
dtype: int64

In [23]:
Tweets_Data_from_Search.isnull().sum()

date          0
content       0
username      0
likes         0
views         0
hashtags    105
url           0
dtype: int64

In [24]:
Himalaya_Wellness_Site_Products_Data.isnull().sum()

title           0
price           3
description    25
url             0
dtype: int64

In [25]:
Himalaya_Wellness_Site_Blog_Data.isnull().sum()

name                 0
posting_timestamp    0
content              0
url                  0
dtype: int64

In [26]:
Facebook_Post_Data.isnull().sum()

posted_by    1
content      3
likes        3
dtype: int64

In [27]:
Himalaya_Youtube_Video_Data.isnull().sum()

link           0
title          0
duration       0
views          0
description    0
upload_date    0
category       0
dtype: int64

In [28]:
Himalaya_Youtube_Video_Search_Data.isnull().sum()

link           0
title          0
duration       0
views          0
description    7
upload_date    0
category       0
dtype: int64

In [29]:
Youtube_Comments_Data.isnull().sum()

video_link     0
author         0
content        0
likes         64
dtype: int64

#### Handling Null Values in Himalaya_Tweets_Data

In [30]:
Himalaya_Tweets_Data['views'].fillna(Himalaya_Tweets_Data[Himalaya_Tweets_Data['views'].isnull()]['likes'].to_dict(),inplace=True)

In [31]:
Himalaya_Tweets_Data.isnull().sum()

date         0
content      0
username     0
likes        0
views        0
hashtags    25
url          0
dtype: int64

In [32]:
Himalaya_Tweets_Data[Himalaya_Tweets_Data['hashtags'].isnull()]

,date,content,username,likes,views,hashtags,url
2,2023-01-17,"@MKC1_Reddy Hi Reddy, Could you please DM us y...",HimalayaIndia,0,29.0,NaN,https://twitter.com/HimalayaIndia/status/16152...
5,2023-01-02,@kishnpalchauhan Your experience is a priority...,HimalayaIndia,0,46.0,NaN,https://twitter.com/HimalayaIndia/status/16098...
7,2022-12-27,"@rlokeshrao @CDSCO_INDIA_INF Hi Lokesh, we alw...",HimalayaIndia,0,49.0,NaN,https://twitter.com/HimalayaIndia/status/16075...
12,2022-11-23,@DeshBhaktReva Please refer to our official st...,HimalayaIndia,3,3.0,NaN,https://twitter.com/HimalayaIndia/status/15952...
22,2022-10-09,@GA_Pansare Please refer to our official state...,HimalayaIndia,0,0.0,NaN,https://twitter.com/HimalayaIndia/status/15791...
23,2022-10-09,@Sunita1093 Please refer to our official state...,HimalayaIndia,0,0.0,NaN,https://twitter.com/HimalayaIndia/status/15791...
24,2022-10-09,@Jagdish_189 Please refer to our official stat...,HimalayaIndia,0,0.0,NaN,https://twitter.com/HimalayaIndia/status/15791...
26,2022-10-06,@SharadG07211098 Hi Sharad! Please be assured ...,HimalayaIndia,0,0.0,NaN,https://twitter.com/HimalayaIndia/status/15778...
29,2022-09-22,"@NVishist17 Hi Nikhil, Please write to us at c...",HimalayaIndia,0,0.0,NaN,https://twitter.com/HimalayaIndia/status/15728...
33,2022-09-05,"@ivnsraju65 Hi Raju, Please refer to our offic...",HimalayaIndia,0,0.0,NaN,https://twitter.com/HimalayaIndia/status/15667...


In [33]:
Himalaya_Tweets_Data['hashtags']=Himalaya_Tweets_Data['hashtags'].fillna("HimalayaIndia")

In [34]:
Himalaya_Tweets_Data.isnull().sum()

date        0
content     0
username    0
likes       0
views       0
hashtags    0
url         0
dtype: int64

#### Handling Null Values in Tweets_Data_from_Search

In [35]:
Tweets_Data_from_Search.isnull().sum()

date          0
content       0
username      0
likes         0
views         0
hashtags    105
url           0
dtype: int64

In [36]:
Tweets_Data_from_Search[Tweets_Data_from_Search['hashtags'].isnull()]['content']

0      @Caudalie\n@HimalayaIndia \n@sleepy\n@babyturc...
1      @PTI_News @PypAyurved @HimalayaIndia @DaburInd...
5      Why are you using "Ashvagandha" instead of "As...
6      @HimalayaIndia \nSam chaudry weekly ladies ko ...
7      @HimalayaIndia ..\nHii sir ...I m from Mumbai....
                             ...                        
122    @HimalayaIndia ma świetne pasty bez fluoru, po...
123    Dept. of Pharm. Chemistry @McopsM organized it...
124    @EduMinOfIndia @HimalayaIndia @dpradhanbjp Res...
125    @dhaval351 @Pampers @Flipkart @jagograhakjago ...
126    @HimalayaIndia Your Liv 52 tablet contains som...
Name: content, Length: 105, dtype: object

In [37]:
Dictionary_of_Word_Frequency=FreqDist(getWordList(Tweets_Data_from_Search[Tweets_Data_from_Search['hashtags'].isnull()]['content']))

In [38]:
Dictionary_of_Word_Frequency

FreqDist({'@HimalayaIndia': 104, 'is': 29, 'and': 19, 'to': 19, 'the': 18, '@theliverdr': 17, 'a': 16, 'of': 15, '@AyurvedaMonk': 15, 'I': 14, ...})

In [39]:
Tweets_Data_from_Search['hashtags']=Tweets_Data_from_Search['hashtags'].fillna("HimalayaIndia")

In [40]:
Tweets_Data_from_Search.isnull().sum()

date        0
content     0
username    0
likes       0
views       0
hashtags    0
url         0
dtype: int64

#### Handling Null Values in Himalaya_Wellness_Site_Products_Data

In [41]:
Himalaya_Wellness_Site_Products_Data.isnull().sum()

title           0
price           3
description    25
url             0
dtype: int64

In [42]:
Himalaya_Wellness_Site_Products_Data[Himalaya_Wellness_Site_Products_Data['price'].isnull()]['url']

726    https://himalayawellness.in/collections/all/pr...
731    https://himalayawellness.in/collections/all/pr...
801    https://himalayawellness.in/collections/all/pr...
Name: url, dtype: object

In [43]:
price_filling_array={
    726:230.00,
    731:130.00,
    801:65.00
}

In [44]:
Himalaya_Wellness_Site_Products_Data['price'].fillna(price_filling_array,inplace=True)
# = Himalaya_Wellness_Site_Products_Data['price'].fillna(price_filling_array)

In [45]:
Himalaya_Wellness_Site_Products_Data['price']=Himalaya_Wellness_Site_Products_Data['price'].apply(
    lambda x:str(x).split()[-1]
)

In [46]:
Himalaya_Wellness_Site_Products_Data.isnull().sum()

title           0
price           0
description    25
url             0
dtype: int64

In [47]:
Himalaya_Wellness_Site_Products_Data['description'].fillna(Himalaya_Wellness_Site_Products_Data[Himalaya_Wellness_Site_Products_Data['description'].isnull()]['title'].to_dict(),inplace=True)

In [48]:
Himalaya_Wellness_Site_Products_Data.isnull().sum()

title          0
price          0
description    0
url            0
dtype: int64

#### Handling Null Values in Himalaya_Wellness_Site_Blog_Data

In [49]:
Himalaya_Wellness_Site_Blog_Data['date']=Himalaya_Wellness_Site_Blog_Data['posting_timestamp'].apply(
    lambda x:x.split()[0])

In [50]:
Himalaya_Wellness_Site_Blog_Data.drop('posting_timestamp',axis=1,inplace=True)

#### Handling Null Values in Facebook_Post_Data

In [51]:
Facebook_Post_Data=Facebook_Post_Data.dropna()

In [52]:
Facebook_Post_Data.isnull().sum()

posted_by    0
content      0
likes        0
dtype: int64

#### Handling Null Values in Himalaya_Youtube_Video_Search_Data

In [53]:
Himalaya_Youtube_Video_Search_Data.isnull().sum()

link           0
title          0
duration       0
views          0
description    7
upload_date    0
category       0
dtype: int64

In [54]:
Himalaya_Youtube_Video_Search_Data[Himalaya_Youtube_Video_Search_Data['description'].isnull()]['title'].to_dict()

{8: '#Himalaya face wash..#Himalaya(clarina) anti-acne cream... best product✌',
 24: 'Himalaya !! products !!skin care ❤️',
 40: 'Nykaa Products Unboxing, Himalaya, Plum Skincare Products Unboxing, Nykaa Makeup Products Unboxing.',
 61: '#Himalaya Aloe Vera Face Wash Amazing Product #shorts',
 63: 'Himalaya cleansing milk( review) deep pore cleansing product for all type of skin',
 96: 'Himalaya skin care product under 150 | Best products in Himalaya |',
 163: 'BEST HIMALAYA PRODUCTS FOR OILY SKIN'}

In [55]:
Himalaya_Youtube_Video_Search_Data['description'].fillna(
Himalaya_Youtube_Video_Search_Data[Himalaya_Youtube_Video_Search_Data['description'].isnull()]['title'].to_dict(),
inplace=True
)

In [56]:
Himalaya_Youtube_Video_Search_Data.isnull().sum()

link           0
title          0
duration       0
views          0
description    0
upload_date    0
category       0
dtype: int64

#### Handling Null Values in Youtube_Comments_Data

In [57]:
Youtube_Comments_Data.fillna(0,inplace=True)

In [58]:
Youtube_Comments_Data.isnull().sum()

video_link    0
author        0
content       0
likes         0
dtype: int64

### Handaling Noisy Data

In [59]:
Himalaya_Tweets_Data['content']=Himalaya_Tweets_Data['content'].apply(lambda x:" ".join(cleaning_of_sentence(x)))

In [60]:
Himalaya_Tweets_Data['hashtags']=Himalaya_Tweets_Data['hashtags'].apply(lambda x:" ".join(cleaning_of_sentence(x)))

In [61]:
Tweets_Data_from_Search['content']=Tweets_Data_from_Search['content'].apply(lambda x:" ".join(cleaning_of_sentence(x)))

In [62]:
Tweets_Data_from_Search['hashtags']=Tweets_Data_from_Search['hashtags'].apply(lambda x:" ".join(cleaning_of_sentence(x)))

In [63]:
Himalaya_Wellness_Site_Products_Data['title']=Himalaya_Wellness_Site_Products_Data['title'].apply(lambda x:" ".join(cleaning_of_sentence(x)))

In [64]:
Himalaya_Wellness_Site_Products_Data['description']=Himalaya_Wellness_Site_Products_Data['description'].apply(lambda x:" ".join(cleaning_of_sentence(x)))

In [112]:
Himalaya_Wellness_Site_Products_Data['price']=Himalaya_Wellness_Site_Products_Data['price'].apply(lambda x:float("".join(x.split(","))))

In [66]:
Himalaya_Wellness_Site_Blog_Data['name']=Himalaya_Wellness_Site_Blog_Data['name'].apply(lambda x:" ".join(cleaning_of_sentence(x)))

In [67]:
Himalaya_Wellness_Site_Blog_Data['content']=Himalaya_Wellness_Site_Blog_Data['content'].apply(lambda x:" ".join(cleaning_of_sentence(x)))

In [68]:
Facebook_Post_Data['posted_by']=Facebook_Post_Data['posted_by'].apply(lambda x:" ".join(cleaning_of_sentence(x)))

In [69]:
Facebook_Post_Data['content']=Facebook_Post_Data['content'].apply(lambda x:" ".join(cleaning_of_sentence(x)))

In [70]:
Himalaya_Youtube_Video_Search_Data['title']=Himalaya_Youtube_Video_Search_Data['title'].apply(lambda x:" ".join(cleaning_of_sentence(x)))

In [71]:
Himalaya_Youtube_Video_Search_Data['description']=Himalaya_Youtube_Video_Search_Data['description'].apply(lambda x:" ".join(cleaning_of_sentence(x)))

In [72]:
Himalaya_Youtube_Video_Search_Data['category']=Himalaya_Youtube_Video_Search_Data['category'].apply(lambda x:" ".join(cleaning_of_sentence(x)))

In [73]:
Himalaya_Youtube_Video_Data['title']=Himalaya_Youtube_Video_Data['title'].apply(lambda x:" ".join(cleaning_of_sentence(x)))

In [74]:
Himalaya_Youtube_Video_Data['description']=Himalaya_Youtube_Video_Data['description'].apply(lambda x:" ".join(cleaning_of_sentence(x)))

In [75]:
Himalaya_Youtube_Video_Data['category']=Himalaya_Youtube_Video_Data['category'].apply(lambda x:" ".join(cleaning_of_sentence(x)))

In [76]:
Youtube_Comments_Data['author']=Youtube_Comments_Data['author'].apply(lambda x:" ".join(cleaning_of_sentence(x)))

In [77]:
Youtube_Comments_Data['content']=Youtube_Comments_Data['content'].apply(lambda x:" ".join(cleaning_of_sentence(x)))

## 2. Data Reduction

### Data Size Reduction

In [78]:
sum(Himalaya_Tweets_Data.duplicated())

0

In [79]:
sum(Tweets_Data_from_Search.duplicated())

0

In [80]:
sum(Himalaya_Wellness_Site_Products_Data.duplicated())

468

In [81]:
Himalaya_Wellness_Site_Products_Data.drop_duplicates(inplace=True)

In [82]:
Himalaya_Wellness_Site_Blog_Data.duplicated().sum()

0

In [83]:
Facebook_Post_Data.duplicated().sum()

2

In [84]:
Facebook_Post_Data.drop_duplicates(inplace=True)

In [85]:
Himalaya_Youtube_Video_Search_Data.duplicated().sum()

0

In [86]:
Himalaya_Youtube_Video_Data.duplicated().sum()

0

In [87]:
Youtube_Comments_Data.duplicated().sum()

0

## 3. Data Transformation

### 1. Data Attribution

In [88]:
Himalaya_Tweets_Data['likes_views_ratio']=getLikePerViews(Himalaya_Tweets_Data['likes'],Himalaya_Tweets_Data['views'])

In [89]:
Himalaya_Tweets_Data

,date,content,username,likes,views,hashtags,url,likes_views_ratio
0,2023-01-25,the himalayawellnesscompany commemorates the s...,HimalayaIndia,13,1052.0,himalayawellnesscompany republicday happyrepub...,https://twitter.com/HimalayaIndia/status/16181...,0.012357
1,2023-01-23,himalaya new manufacturing facility in tumakur...,HimalayaIndia,20,1284.0,iso safety himalayawellness,https://twitter.com/HimalayaIndia/status/16173...,0.015576
2,2023-01-17,mkc1 reddy hi reddy could you please dm your c...,HimalayaIndia,0,29.0,himalayaindia,https://twitter.com/HimalayaIndia/status/16152...,0.000000
3,2023-01-13,let celebrate the onset of warmer day and pros...,HimalayaIndia,3,473.0,himalayawellnesscompany lohri pongal makarsank...,https://twitter.com/HimalayaIndia/status/16138...,0.006342
4,2023-01-13,himalaya drive initiative to enrich life we ha...,HimalayaIndia,3,679.0,himalaya government school,https://twitter.com/HimalayaIndia/status/16137...,0.004418
...,...,...,...,...,...,...,...,...
95,2022-03-24,samarsanatan the social medium post circulated...,HimalayaIndia,0,0.0,himalayawellness fake,https://twitter.com/HimalayaIndia/status/15068...,0.000000
96,2022-03-22,bhanu the social medium post circulated about ...,HimalayaIndia,1,1.0,himalayawellness fake,https://twitter.com/HimalayaIndia/status/15062...,1.000000
97,2022-03-22,thanush hi shree please share your contact det...,HimalayaIndia,1,1.0,himalayaindia,https://twitter.com/HimalayaIndia/status/15061...,1.000000
98,2022-03-21,hardik rathod this doe not sound right hardik ...,HimalayaIndia,0,0.0,himalayaindia,https://twitter.com/HimalayaIndia/status/15057...,0.000000


In [90]:
Tweets_Data_from_Search['likes_views_ratio']=getLikePerViews(Tweets_Data_from_Search['likes'],Tweets_Data_from_Search['views'])

In [91]:
Tweets_Data_from_Search

,date,content,username,likes,views,hashtags,url,likes_views_ratio
0,2023-02-12,caudalie himalayaindia sleepy babyturcotr deprem,rabiaerTURKc1,0,61,himalayaindia,https://twitter.com/rabiaerTURKc1/status/16248...,0.000000
1,2023-02-12,pti news pypayurved himalayaindia daburindia z...,Green_Laalten,0,183,himalayaindia,https://twitter.com/Green_Laalten/status/16245...,0.000000
2,2023-02-10,please declare these product bhagwantmann cert...,azeemali3119,2,115,punjab ayurveda unani patanjalifoods patanjali...,https://twitter.com/azeemali3119/status/162389...,0.017391
3,2023-02-08,the price of soap are rising like anything in ...,AmericanLove4u,0,58,soap india ausvsind cricket indvsaus,https://twitter.com/AmericanLove4u/status/1623...,0.000000
4,2023-02-08,shop himalayaindia herbal product from iherb f...,ayurvedanelly,0,49,india himalaya himalayaindia himalayawellness,https://twitter.com/ayurvedanelly/status/16232...,0.000000
...,...,...,...,...,...,...,...,...
122,2023-01-03,himalayaindia ma świetne pasty bez fluoru pole...,severiaan,0,29,himalayaindia,https://twitter.com/severiaan/status/161033390...,0.000000
123,2023-01-03,dept of pharm chemistry mcopsm organized it se...,McopsM,2,140,himalayaindia,https://twitter.com/McopsM/status/161023740102...,0.014286
124,2023-01-03,eduminofindia himalayaindia dpradhanbjp respec...,HemanthSharma11,0,47,himalayaindia,https://twitter.com/HemanthSharma11/status/161...,0.000000
125,2023-01-02,dhaval pampers flipkart jagograhakjago huggies...,gourav00111,0,25,himalayaindia,https://twitter.com/gourav00111/status/1609924...,0.000000


## 4. Data Integration

In [92]:
Whole_Youtube_Video_Data=pandas.concat([Himalaya_Youtube_Video_Data,Himalaya_Youtube_Video_Search_Data]).reset_index(drop=True)

In [93]:
Whole_Youtube_Video_Data

,link,title,duration,views,description,upload_date,category
0,https://www.youtube.com/watch?v=DAIFzQy3DMs,himalaya youth eternity face sheet mask health...,20,129,himalaya youth eternity face sheet mask are in...,2023-02-17,howto style
1,https://www.youtube.com/watch?v=vvXGQWoLA7Q,himalaya youth eternity face sheet mask health...,50,191,himalaya youth eternity face sheet mask are in...,2023-02-17,howto style
2,https://www.youtube.com/watch?v=Et8IRYf_iS0,complete gum care protection,11,47165,give your gum complete natural care with herb ...,2023-01-23,howto style
3,https://www.youtube.com/watch?v=VogJG0BsbQw,himalaya ayurveda gum care,10,15210,wipe out gum problem with goodness of nature t...,2023-01-23,howto style
4,https://www.youtube.com/watch?v=RFpxEatO5r0,cure bleeding gum with himalaya ayurveda gum care,10,67714,cure bleeding from the root himalaya ayurveda ...,2023-01-23,howto style
...,...,...,...,...,...,...,...
270,https://www.youtube.com/watch?v=KsMFhXFvMxA,himalaya purifying neem face wash kannada,60,188249,himalaya purifying neem face wash is clinicall...,2020-07-03,howto style
271,https://www.youtube.com/watch?v=wPxJGlblkPs,himalaya purifying neem face wash tamil,60,439263,himalaya purifying neem face wash is clinicall...,2020-07-03,howto style
272,https://www.youtube.com/watch?v=er86teCnoVw,summer skincare routine under using only himal...,342,596,skincare routine under using only himalya prod...,2020-07-01,howto style
273,https://www.youtube.com/watch?v=L5T943418Ko,himalaya baby soap and himalaya baby product r...,557,2555,hai friend welcome to my channel today am shar...,2020-06-30,howto style


In [94]:
Whole_Twitter_Data=pandas.concat([Himalaya_Tweets_Data,Tweets_Data_from_Search],ignore_index=True)

In [95]:
Whole_Twitter_Data

,date,content,username,likes,views,hashtags,url,likes_views_ratio
0,2023-01-25,the himalayawellnesscompany commemorates the s...,HimalayaIndia,13,1052.0,himalayawellnesscompany republicday happyrepub...,https://twitter.com/HimalayaIndia/status/16181...,0.012357
1,2023-01-23,himalaya new manufacturing facility in tumakur...,HimalayaIndia,20,1284.0,iso safety himalayawellness,https://twitter.com/HimalayaIndia/status/16173...,0.015576
2,2023-01-17,mkc1 reddy hi reddy could you please dm your c...,HimalayaIndia,0,29.0,himalayaindia,https://twitter.com/HimalayaIndia/status/16152...,0.000000
3,2023-01-13,let celebrate the onset of warmer day and pros...,HimalayaIndia,3,473.0,himalayawellnesscompany lohri pongal makarsank...,https://twitter.com/HimalayaIndia/status/16138...,0.006342
4,2023-01-13,himalaya drive initiative to enrich life we ha...,HimalayaIndia,3,679.0,himalaya government school,https://twitter.com/HimalayaIndia/status/16137...,0.004418
...,...,...,...,...,...,...,...,...
222,2023-01-03,himalayaindia ma świetne pasty bez fluoru pole...,severiaan,0,29.0,himalayaindia,https://twitter.com/severiaan/status/161033390...,0.000000
223,2023-01-03,dept of pharm chemistry mcopsm organized it se...,McopsM,2,140.0,himalayaindia,https://twitter.com/McopsM/status/161023740102...,0.014286
224,2023-01-03,eduminofindia himalayaindia dpradhanbjp respec...,HemanthSharma11,0,47.0,himalayaindia,https://twitter.com/HemanthSharma11/status/161...,0.000000
225,2023-01-02,dhaval pampers flipkart jagograhakjago huggies...,gourav00111,0,25.0,himalayaindia,https://twitter.com/gourav00111/status/1609924...,0.000000


## 5. Data Filtaration

In [170]:
Whole_Twitter_Data[Whole_Twitter_Data['views']>Whole_Twitter_Data['views'].mean()].head()

,date,content,username,likes,views,hashtags,url,likes_views_ratio
0,2023-01-25,the himalayawellnesscompany commemorates the s...,HimalayaIndia,13,1052.0,himalayawellnesscompany republicday happyrepub...,https://twitter.com/HimalayaIndia/status/16181...,0.012357
1,2023-01-23,himalaya new manufacturing facility in tumakur...,HimalayaIndia,20,1284.0,iso safety himalayawellness,https://twitter.com/HimalayaIndia/status/16173...,0.015576
3,2023-01-13,let celebrate the onset of warmer day and pros...,HimalayaIndia,3,473.0,himalayawellnesscompany lohri pongal makarsank...,https://twitter.com/HimalayaIndia/status/16138...,0.006342
4,2023-01-13,himalaya drive initiative to enrich life we ha...,HimalayaIndia,3,679.0,himalaya government school,https://twitter.com/HimalayaIndia/status/16137...,0.004418
6,2022-12-31,let bid adieu to and enter with new opportunit...,HimalayaIndia,15,852.0,happynewyear,https://twitter.com/HimalayaIndia/status/16090...,0.017606


In [161]:
Whole_Twitter_Data.sort_values(by='likes_views_ratio',ascending=False).head(10)

,date,content,username,likes,views,hashtags,url,likes_views_ratio
51,2022-07-25,himalayawellness celebrates and honor year of ...,HimalayaIndia,22,22.0,himalayawellness partnership partnersinprogres...,https://twitter.com/HimalayaIndia/status/15514...,1.0
62,2022-07-04,hindi hun main the social medium post circulat...,HimalayaIndia,1,1.0,himalayawellness fake,https://twitter.com/HimalayaIndia/status/15437...,1.0
59,2022-07-05,hi this is not authentic and we request you to...,HimalayaIndia,1,1.0,fake,https://twitter.com/HimalayaIndia/status/15442...,1.0
55,2022-07-18,we have won economictimes sustainable organiza...,HimalayaIndia,28,28.0,sustainability himalayawellness,https://twitter.com/HimalayaIndia/status/15489...,1.0
54,2022-07-18,mdsarfa hi this is not authentic and we reques...,HimalayaIndia,1,1.0,fake,https://twitter.com/HimalayaIndia/status/15489...,1.0
52,2022-07-21,himalayawellness introduces partnersinprogress...,HimalayaIndia,22,22.0,himalayawellness partnersinprogress diverse in...,https://twitter.com/HimalayaIndia/status/15500...,1.0
89,2022-04-01,sirpareshrawal thank you sirpareshrawal we don...,HimalayaIndia,1951,1951.0,himalaya sharmajinamkeen,https://twitter.com/HimalayaIndia/status/15097...,1.0
49,2022-08-01,crescent packaging ha played key role in our p...,HimalayaIndia,13,13.0,partnersinprogress packaging himalayawellness,https://twitter.com/HimalayaIndia/status/15540...,1.0
48,2022-08-08,himalaya incredible year of association with p...,HimalayaIndia,19,19.0,partnersinprogress himalayawellness packaging ...,https://twitter.com/HimalayaIndia/status/15565...,1.0
90,2022-04-01,,HimalayaIndia,6653,6653.0,himalayaindia,https://twitter.com/HimalayaIndia/status/15097...,1.0


In [169]:
Whole_Youtube_Video_Data.head(10).sort_values(by="views",ascending=False)

,link,title,duration,views,description,upload_date,category
5,https://www.youtube.com/watch?v=x7U9HMy2sp8,himalaya pain balm strong keep calm use himala...,35,1630051,when you have headache you tend to become head...,2022-11-14,howto style
6,https://www.youtube.com/watch?v=1qGuali2VUU,himalaya pain balm strong keep calm use himala...,20,1185314,when you have headache you tend to become head...,2022-11-14,howto style
7,https://www.youtube.com/watch?v=GGqfYoq-mO0,himalaya pain balm strong keep calm use himala...,10,1184439,when you have headache you tend to become head...,2022-11-14,howto style
4,https://www.youtube.com/watch?v=RFpxEatO5r0,cure bleeding gum with himalaya ayurveda gum care,10,67714,cure bleeding from the root himalaya ayurveda ...,2023-01-23,howto style
2,https://www.youtube.com/watch?v=Et8IRYf_iS0,complete gum care protection,11,47165,give your gum complete natural care with herb ...,2023-01-23,howto style
3,https://www.youtube.com/watch?v=VogJG0BsbQw,himalaya ayurveda gum care,10,15210,wipe out gum problem with goodness of nature t...,2023-01-23,howto style
8,https://www.youtube.com/watch?v=vBgbsIC_AoA,himalaya pain balm strong keep calm use himala...,6,8321,when you have headache you tend to become head...,2022-11-14,howto style
9,https://www.youtube.com/watch?v=sP-wHgIJ3gs,pimple problem and still using soap now no mor...,15,6519,science say that soap can increase pimple and ...,2022-11-10,howto style
1,https://www.youtube.com/watch?v=vvXGQWoLA7Q,himalaya youth eternity face sheet mask health...,50,191,himalaya youth eternity face sheet mask are in...,2023-02-17,howto style
0,https://www.youtube.com/watch?v=DAIFzQy3DMs,himalaya youth eternity face sheet mask health...,20,129,himalaya youth eternity face sheet mask are in...,2023-02-17,howto style


# Data Saving on Database

In [117]:
Whole_Twitter_Data.reset_index(drop=True,inplace=True)
Whole_Youtube_Video_Data.reset_index(drop=True,inplace=True)
Youtube_Comments_Data.reset_index(drop=True,inplace=True)
Facebook_Post_Data.reset_index(drop=True,inplace=True)
Himalaya_Wellness_Site_Blog_Data.reset_index(drop=True,inplace=True)
Himalaya_Wellness_Site_Products_Data.reset_index(drop=True,inplace=True)

## Saving in CSV Files

In [118]:
Whole_Twitter_Data.to_csv("../Data/Processesed_Data/Whole_Twitter_Data.csv",index=False)

In [119]:
Whole_Youtube_Video_Data.to_csv("../Data/Processesed_Data/Whole_Youtube_Video_Data.csv",index=False)

In [120]:
Youtube_Comments_Data.to_csv("../Data/Processesed_Data/Youtube_Comments_Data.csv",index=False)

In [121]:
Facebook_Post_Data.to_csv("../Data/Processesed_Data/Facebook_Post_Data.csv",index=False)

In [122]:
Himalaya_Wellness_Site_Blog_Data.to_csv("../Data/Processesed_Data/Himalaya_Wellness_Site_Blog_Data.csv",index=False)

In [123]:
Himalaya_Wellness_Site_Products_Data.to_csv("../Data/Processesed_Data/Himalaya_Wellness_Site_Products_Data.csv",index=False)

## Data Saving on Non-Relational Database


In [124]:
mongodb_connection.list_database_names()

['admin', 'local']

In [134]:
social_media_collected_database=mongodb_connection['social_media_collected_database']

In [142]:
def insertInCollection(Data,collection_name):
    collection=social_media_collected_database[collection_name]
    for row in range(len(Data)):
        collection.insert_one(Data.iloc[row,:].to_dict())
        print("row done:",row)

### Document Saving

In [143]:
insertInCollection(Himalaya_Wellness_Site_Blog_Data,'himalaya_site_blogs_collection')

row done: 0
row done: 1
row done: 2
row done: 3
row done: 4
row done: 5
row done: 6
row done: 7
row done: 8
row done: 9
row done: 10
row done: 11
row done: 12
row done: 13
row done: 14
row done: 15
row done: 16
row done: 17
row done: 18
row done: 19


In [144]:
insertInCollection(Whole_Twitter_Data,'twitter_data_collection')

row done: 0
row done: 1
row done: 2
row done: 3
row done: 4
row done: 5
row done: 6
row done: 7
row done: 8
row done: 9
row done: 10
row done: 11
row done: 12
row done: 13
row done: 14
row done: 15
row done: 16
row done: 17
row done: 18
row done: 19
row done: 20
row done: 21
row done: 22
row done: 23
row done: 24
row done: 25
row done: 26
row done: 27
row done: 28
row done: 29
row done: 30
row done: 31
row done: 32
row done: 33
row done: 34
row done: 35
row done: 36
row done: 37
row done: 38
row done: 39
row done: 40
row done: 41
row done: 42
row done: 43
row done: 44
row done: 45
row done: 46
row done: 47
row done: 48
row done: 49
row done: 50
row done: 51
row done: 52
row done: 53
row done: 54
row done: 55
row done: 56
row done: 57
row done: 58
row done: 59
row done: 60
row done: 61
row done: 62
row done: 63
row done: 64
row done: 65
row done: 66
row done: 67
row done: 68
row done: 69
row done: 70
row done: 71
row done: 72
row done: 73
row done: 74
row done: 75
row done: 76
row done:

In [145]:
insertInCollection(Whole_Youtube_Video_Data,'youtube_videos_data_collection')

row done: 0
row done: 1
row done: 2
row done: 3
row done: 4
row done: 5
row done: 6
row done: 7
row done: 8
row done: 9
row done: 10
row done: 11
row done: 12
row done: 13
row done: 14
row done: 15
row done: 16
row done: 17
row done: 18
row done: 19
row done: 20
row done: 21
row done: 22
row done: 23
row done: 24
row done: 25
row done: 26
row done: 27
row done: 28
row done: 29
row done: 30
row done: 31
row done: 32
row done: 33
row done: 34
row done: 35
row done: 36
row done: 37
row done: 38
row done: 39
row done: 40
row done: 41
row done: 42
row done: 43
row done: 44
row done: 45
row done: 46
row done: 47
row done: 48
row done: 49
row done: 50
row done: 51
row done: 52
row done: 53
row done: 54
row done: 55
row done: 56
row done: 57
row done: 58
row done: 59
row done: 60
row done: 61
row done: 62
row done: 63
row done: 64
row done: 65
row done: 66
row done: 67
row done: 68
row done: 69
row done: 70
row done: 71
row done: 72
row done: 73
row done: 74
row done: 75
row done: 76
row done:

In [146]:
insertInCollection(Youtube_Comments_Data,'youtube_comments_data_collection')

row done: 0
row done: 1
row done: 2
row done: 3
row done: 4
row done: 5
row done: 6
row done: 7
row done: 8
row done: 9
row done: 10
row done: 11
row done: 12
row done: 13
row done: 14
row done: 15
row done: 16
row done: 17
row done: 18
row done: 19
row done: 20
row done: 21
row done: 22
row done: 23
row done: 24
row done: 25
row done: 26
row done: 27
row done: 28
row done: 29
row done: 30
row done: 31
row done: 32
row done: 33
row done: 34
row done: 35
row done: 36
row done: 37
row done: 38
row done: 39
row done: 40
row done: 41
row done: 42
row done: 43
row done: 44
row done: 45
row done: 46
row done: 47
row done: 48
row done: 49
row done: 50
row done: 51
row done: 52
row done: 53
row done: 54
row done: 55
row done: 56
row done: 57
row done: 58
row done: 59
row done: 60
row done: 61
row done: 62
row done: 63
row done: 64
row done: 65
row done: 66
row done: 67
row done: 68
row done: 69
row done: 70
row done: 71
row done: 72
row done: 73
row done: 74
row done: 75
row done: 76
row done:

In [147]:
insertInCollection(Himalaya_Wellness_Site_Products_Data,'himalaya_site_products_data_collection')

row done: 0
row done: 1
row done: 2
row done: 3
row done: 4
row done: 5
row done: 6
row done: 7
row done: 8
row done: 9
row done: 10
row done: 11
row done: 12
row done: 13
row done: 14
row done: 15
row done: 16
row done: 17
row done: 18
row done: 19
row done: 20
row done: 21
row done: 22
row done: 23
row done: 24
row done: 25
row done: 26
row done: 27
row done: 28
row done: 29
row done: 30
row done: 31
row done: 32
row done: 33
row done: 34
row done: 35
row done: 36
row done: 37
row done: 38
row done: 39
row done: 40
row done: 41
row done: 42
row done: 43
row done: 44
row done: 45
row done: 46
row done: 47
row done: 48
row done: 49
row done: 50
row done: 51
row done: 52
row done: 53
row done: 54
row done: 55
row done: 56
row done: 57
row done: 58
row done: 59
row done: 60
row done: 61
row done: 62
row done: 63
row done: 64
row done: 65
row done: 66
row done: 67
row done: 68
row done: 69
row done: 70
row done: 71
row done: 72
row done: 73
row done: 74
row done: 75
row done: 76
row done:

In [148]:
insertInCollection(Facebook_Post_Data,'facebook_data_collection')

row done: 0
row done: 1
row done: 2
row done: 3
row done: 4
row done: 5
row done: 6
row done: 7
row done: 8
row done: 9
row done: 10
row done: 11
row done: 12
row done: 13
row done: 14
row done: 15
row done: 16
row done: 17
row done: 18
row done: 19
row done: 20
row done: 21
row done: 22
row done: 23
row done: 24
row done: 25
row done: 26
row done: 27
row done: 28
row done: 29
row done: 30
row done: 31
row done: 32
row done: 33
row done: 34
row done: 35
row done: 36
row done: 37
row done: 38
row done: 39
row done: 40
row done: 41
row done: 42
row done: 43
row done: 44
row done: 45
row done: 46
row done: 47
row done: 48
row done: 49
row done: 50
row done: 51
row done: 52
row done: 53
row done: 54
row done: 55
row done: 56
row done: 57
row done: 58
row done: 59
row done: 60


## Data Saving on Relational Database

### SQL QUERIES

In [104]:
INITIAL_SQL_QUERY=["""
CREATE DATABASE social_media_collected_data;
""","""
use social_media_collected_data;
""","""
CREATE TABLE twitter_data(
    `id` int NOT NULL PRIMARY KEY AUTO_INCREMENT,
    `date` text,
    `content` text,
    `username` text,
    `likes` int,
    `views` int,
    `hashtags` text,
    `url` text,
    `likes_views_ratio` float
);

""","""

CREATE TABLE himalaya_site_products_data(
    `id` int NOT NULL PRIMARY KEY AUTO_INCREMENT,
    `title` text,
    `price` float,
    `description` text,
    `url` text
);
""","""

CREATE TABLE himalaya_site_blogs_data(
    `id` int NOT NULL PRIMARY KEY AUTO_INCREMENT,
    `name` text,
    `content` text,
    `url` text,
    `date` text
);

""","""

CREATE TABLE facebook_data(
    `id` int NOT NULL PRIMARY KEY AUTO_INCREMENT,
    `posted_by` text,
    `content` text,
    `likes` float
);
""","""

CREATE TABLE youtube_videos_data(
    `id` int NOT NULL PRIMARY KEY AUTO_INCREMENT,
    `link` text,
    `title` text,
    `duration` text,
    `views` text,
    `description` text,
    `upload_date` text,
    `category` text
);
""","""

CREATE TABLE youtube_comments_data(
    `id` int NOT NULL PRIMARY KEY AUTO_INCREMENT,
    `video_link` text,
    `author` text,
    `content` text,
    `likes` float
);
"""]

In [105]:
def initial_setup():
    try:
        queryExecuter.execute("use social_media_collected_data;")
        print("Data base Exists")
    except Exception as e:
        print(e)
        print("Creating Database ......")
        for query in INITIAL_SQL_QUERY:
            print(query)
            queryExecuter.execute(query)
        mysql_connection.commit()

In [106]:
initial_setup()

Data base Exists


In [127]:
def insertDataFromDataFrame(Data:pandas.DataFrame,database_name)->None:
    try:
        for row in range(len(Data)):
            SQL_QUERY="INSERT INTO "+database_name+"("+",".join(list(Data.columns))+")VALUES("
            for column in Data.columns:
                if list(Data.columns).index(column)!=0:
                    SQL_QUERY=SQL_QUERY+","
                try:
                    int(Data[column][row])
                    SQL_QUERY=SQL_QUERY+f"{Data[column][row]}"
                except:
                    SQL_QUERY=SQL_QUERY+f"'{Data[column][row]}'"
            SQL_QUERY=SQL_QUERY+");"
            queryExecuter.execute(SQL_QUERY)
            print("row done:",row)
        mysql_connection.commit()
    except Exception as e:
        print(e)
        mysql_connection.rollback()

### Data Loading

In [590]:
insertDataFromDataFrame(Himalaya_Wellness_Site_Blog_Data,"himalaya_site_blogs_data")

row done: 0
row done: 1
row done: 2
row done: 3
row done: 4
row done: 5
row done: 6
row done: 7
row done: 8
row done: 9
row done: 10
row done: 11
row done: 12
row done: 13
row done: 14
row done: 15
row done: 16
row done: 17
row done: 18
row done: 19


In [591]:
insertDataFromDataFrame(Whole_Twitter_Data,"twitter_data")

row done: 0
row done: 1
row done: 2
row done: 3
row done: 4
row done: 5
row done: 6
row done: 7
row done: 8
row done: 9
row done: 10
row done: 11
row done: 12
row done: 13
row done: 14
row done: 15
row done: 16
row done: 17
row done: 18
row done: 19
row done: 20
row done: 21
row done: 22
row done: 23
row done: 24
row done: 25
row done: 26
row done: 27
row done: 28
row done: 29
row done: 30
row done: 31
row done: 32
row done: 33
row done: 34
row done: 35
row done: 36
row done: 37
row done: 38
row done: 39
row done: 40
row done: 41
row done: 42
row done: 43
row done: 44
row done: 45
row done: 46
row done: 47
row done: 48
row done: 49
row done: 50
row done: 51
row done: 52
row done: 53
row done: 54
row done: 55
row done: 56
row done: 57
row done: 58
row done: 59
row done: 60
row done: 61
row done: 62
row done: 63
row done: 64
row done: 65
row done: 66
row done: 67
row done: 68
row done: 69
row done: 70
row done: 71
row done: 72
row done: 73
row done: 74
row done: 75
row done: 76
row done:

In [128]:
insertDataFromDataFrame(Himalaya_Wellness_Site_Products_Data,"himalaya_site_products_data")

row done: 0
row done: 1
row done: 2
row done: 3
row done: 4
row done: 5
row done: 6
row done: 7
row done: 8
row done: 9
row done: 10
row done: 11
row done: 12
row done: 13
row done: 14
row done: 15
row done: 16
row done: 17
row done: 18
row done: 19
row done: 20
row done: 21
row done: 22
row done: 23
row done: 24
row done: 25
row done: 26
row done: 27
row done: 28
row done: 29
row done: 30
row done: 31
row done: 32
row done: 33
row done: 34
row done: 35
row done: 36
row done: 37
row done: 38
row done: 39
row done: 40
row done: 41
row done: 42
row done: 43
row done: 44
row done: 45
row done: 46
row done: 47
row done: 48
row done: 49
row done: 50
row done: 51
row done: 52
row done: 53
row done: 54
row done: 55
row done: 56
row done: 57
row done: 58
row done: 59
row done: 60
row done: 61
row done: 62
row done: 63
row done: 64
row done: 65
row done: 66
row done: 67
row done: 68
row done: 69
row done: 70
row done: 71
row done: 72
row done: 73
row done: 74
row done: 75
row done: 76
row done:

In [129]:
insertDataFromDataFrame(Facebook_Post_Data,"facebook_data")

row done: 0
row done: 1
row done: 2
row done: 3
row done: 4
row done: 5
row done: 6
row done: 7
row done: 8
row done: 9
row done: 10
row done: 11
row done: 12
row done: 13
row done: 14
row done: 15
row done: 16
row done: 17
row done: 18
row done: 19
row done: 20
row done: 21
row done: 22
row done: 23
row done: 24
row done: 25
row done: 26
row done: 27
row done: 28
row done: 29
row done: 30
row done: 31
row done: 32
row done: 33
row done: 34
row done: 35
row done: 36
row done: 37
row done: 38
row done: 39
row done: 40
row done: 41
row done: 42
row done: 43
row done: 44
row done: 45
row done: 46
row done: 47
row done: 48
row done: 49
row done: 50
row done: 51
row done: 52
row done: 53
row done: 54
row done: 55
row done: 56
row done: 57
row done: 58
row done: 59
row done: 60


In [596]:
insertDataFromDataFrame(Whole_Youtube_Video_Data,"youtube_videos_data")

row done: 0
row done: 1
row done: 2
row done: 3
row done: 4
row done: 5
row done: 6
row done: 7
row done: 8
row done: 9
row done: 10
row done: 11
row done: 12
row done: 13
row done: 14
row done: 15
row done: 16
row done: 17
row done: 18
row done: 19
row done: 20
row done: 21
row done: 22
row done: 23
row done: 24
row done: 25
row done: 26
row done: 27
row done: 28
row done: 29
row done: 30
row done: 31
row done: 32
row done: 33
row done: 34
row done: 35
row done: 36
row done: 37
row done: 38
row done: 39
row done: 40
row done: 41
row done: 42
row done: 43
row done: 44
row done: 45
row done: 46
row done: 47
row done: 48
row done: 49
row done: 50
row done: 51
row done: 52
row done: 53
row done: 54
row done: 55
row done: 56
row done: 57
row done: 58
row done: 59
row done: 60
row done: 61
row done: 62
row done: 63
row done: 64
row done: 65
row done: 66
row done: 67
row done: 68
row done: 69
row done: 70
row done: 71
row done: 72
row done: 73
row done: 74
row done: 75
row done: 76
row done:

In [597]:
insertDataFromDataFrame(Youtube_Comments_Data,"youtube_comments_data")

row done: 0
row done: 1
row done: 2
row done: 3
row done: 4
row done: 5
row done: 6
row done: 7
row done: 8
row done: 9
row done: 10
row done: 11
row done: 12
row done: 13
row done: 14
row done: 15
row done: 16
row done: 17
row done: 18
row done: 19
row done: 20
row done: 21
row done: 22
row done: 23
row done: 24
row done: 25
row done: 26
row done: 27
row done: 28
row done: 29
row done: 30
row done: 31
row done: 32
row done: 33
row done: 34
row done: 35
row done: 36
row done: 37
row done: 38
row done: 39
row done: 40
row done: 41
row done: 42
row done: 43
row done: 44
row done: 45
row done: 46
row done: 47
row done: 48
row done: 49
row done: 50
row done: 51
row done: 52
row done: 53
row done: 54
row done: 55
row done: 56
row done: 57
row done: 58
row done: 59
row done: 60
row done: 61
row done: 62
row done: 63
row done: 64
row done: 65
row done: 66
row done: 67
row done: 68
row done: 69
row done: 70
row done: 71
row done: 72
row done: 73
row done: 74
row done: 75
row done: 76
row done: